This [__IHaskell__](https://github.com/gibiansky/IHaskell) notebook is running with the following environment:

In [1]:
:! ghc --version
:! grep resolver /opt/stack/global-project/stack.yaml
:! ghc-pkg list | grep replace-megaparsec

The Glorious Glasgow Haskell Compilation System, version 8.6.5

resolver: lts-14.27

    replace-megaparsec-1.3.2.0

For more information, see [ihaskell-notebook](https://github.com/jamesdbrock/ihaskell-notebook)

# Usage Examples for [__replace-megaparsec__](https://github.com/jamesdbrock/replace-megaparsec)

In [2]:
import Data.Void
import Replace.Megaparsec
import Text.Megaparsec
import Text.Megaparsec.Char
import Text.Megaparsec.Char.Lexer

## Split strings with [`splitCap`](https://hackage.haskell.org/package/replace-megaparsec/docs/Replace-Megaparsec.html#v:splitCap)

### Find all pattern matches, capture the matched text and the parsed result

Separate the input string into sections which can be parsed as a hexadecimal
number with a prefix `"0x"`, and sections which can't. Parse the numbers.

In [3]:
let hexparser = chunk "0x" *> hexadecimal :: Parsec Void String Integer
splitCap (match hexparser) "0xA 000 0xFFFF"

[Right ("0xA",10),Left " 000 ",Right ("0xFFFF",65535)]

### Find all pattern matches, capture only the locations of the matched patterns

Find all of the sections of the stream which are letters. Capture a list of
the offsets of the beginning of every pattern match.

In [4]:
import Data.Either
let letterOffset = getOffset <* some letterChar :: Parsec Void String Int
rights $ splitCap letterOffset " a  bc "

[1,4]

### Pattern match balanced parentheses

Find groups of balanced nested parentheses. This is an example of a
“context-free” grammar, a pattern that can't be expressed by a regular
expression. We can express the pattern with a recursive parser.

In [5]:
import Data.Functor (void)
import Data.Bifunctor (second)
let parens :: Parsec Void String ()
    parens = do
        char '('
        manyTill
            (void (noneOf "()") <|> void parens)
            (char ')')
        pure ()

second fst <$> splitCap (match parens) "(()) (()())"

[Right "(())",Left " ",Right "(()())"]

## Edit strings with [`streamEdit`](https://hackage.haskell.org/package/replace-megaparsec/docs/Replace-Megaparsec.html#v:streamEdit)

The following examples show how to search for a pattern in a string of text
and then edit the string of text to substitute in some replacement text
for the matched patterns.

### Pattern match and replace with a constant

Replace all carriage-return-newline instances with newline.

In [6]:
let crnl = chunk "\r\n" :: Parsec Void String String
streamEdit crnl (const "\n") "1\r\n2\r\n"

"1\n2\n"

### Pattern match and edit the matches

Replace alphabetic characters with the next character in the alphabet.

In [7]:
let somelet = some letterChar :: Parsec Void String String
streamEdit somelet (fmap succ) "HAL 9000"

"IBM 9000"

### Pattern match and maybe edit the matches, or maybe leave them alone

Find all of the string sections *`s`* which can be parsed as a
hexadecimal number *`r`*,
and if *`r≤16`*, then replace *`s`* with a decimal number. Uses the
[`match`](https://hackage.haskell.org/package/megaparsec/docs/Text-Megaparsec.html#v:match)
combinator.

In [8]:
let hexparser = chunk "0x" *> hexadecimal :: Parsec Void String Integer
streamEdit (match hexparser) (\(s,r) -> if r<=16 then show r else s) "0xA 000 0xFFFF"

"10 000 0xFFFF"

### Pattern match and edit the matches in IO with [`streamEditT`](https://hackage.haskell.org/package/replace-megaparsec/docs/Replace-Megaparsec.html#v:streamEditT)

Find an environment variable in curly braces and replace it with its
value from the environment.

In [9]:
import System.Environment (getEnv)
let bracevar = char '{' *> manyTill anySingle (char '}') :: ParsecT Void String IO String
streamEditT bracevar getEnv "- {HOME} -"

"- /home/jovyan -"

### Context-sensitive pattern match and edit the matches with [`streamEditT`](https://hackage.haskell.org/package/replace-megaparsec/docs/Replace-Megaparsec.html#v:streamEditT)

Capitalize the third letter in a string. The `capThird` parser searches for
individual letters, and it needs to remember how many times it has run so
that it can match successfully only on the third time that it finds a letter.
To enable the parser to remember how many times it has run, we'll
compose the parser with a [`State`](https://hackage.haskell.org/package/mtl/docs/Control-Monad-State-Lazy.html#t:State) monad from
the `mtl` package. Because it has
stateful memory, this parser is an example of a “context-sensitive” grammar.

In [10]:
import qualified Control.Monad.State.Strict as MTL
import Control.Monad.State.Strict (get, put, evalState)
import Data.Char (toUpper)

let capThird :: ParsecT Void String (MTL.State Int) String
    capThird = do
        x <- letterChar
        i <- get
        let i' = i+1
        put i'
        if i'==3 then return [x] else empty

flip evalState 0 $ streamEditT capThird (pure . fmap toUpper) "a a a a a"

"a a A a a"

### Pattern match, edit the matches, and count the edits with [`streamEditT`](https://hackage.haskell.org/package/replace-megaparsec/docs/Replace-Megaparsec.html#v:streamEditT)

Find and capitalize no more than three letters in a string, and return the edited string along with the number of letters capitalized. To enable the editor function to remember how many letters it has capitalized, we'll run `streamEditT` in the `State` monad from the `mtl` package. Use this technique to get the same functionality as Python [`re.subn`](https://docs.python.org/3/library/re.html#re.subn).

In [11]:
import qualified Control.Monad.State.Strict as MTL
import Control.Monad.State.Strict (get, put, runState)
import Data.Char (toUpper)
        
let editThree :: Char -> MTL.State Int String
    editThree x = do
        i <- get
        let i' = i+1
        if i'<=3
            then do
                put i'
                pure [toUpper x]
            else pure [x]

flip runState 0 $ streamEditT letterChar editThree "a a a a a"

("A A A a a",3)